In [1]:
import sys
import sh
import pandas as pd
import re
import json

In [2]:
DATA_DIR = './data/'
ARCHIVE = DATA_DIR+'/archive/'

In [3]:
# !cat scripts/process.sh

In [4]:
# !head -n5 scripts/process.sh > data/archive/untar.sh
# !chmod +x data/archive/untar.sh

In [7]:
sh.rsync('-avz', 'clic_pretoria:code/foodborne-nyc/data/archive/20170503_0[45]*.tar', DATA_DIR+'/archive/')

receiving file list ... done

sent 16 bytes  received 121 bytes  91.33 bytes/sec
total size is 450560  speedup is 3288.76

In [9]:
!cd data/archive && echo $PWD && ./untar.sh && cd - && echo $PWD

/Users/yogi/code/foodborne-nyc/data/archive
+ for f in '*.tar'
+ p='*'
+ gtar '--xform=s/.*\/\(.*\)/*_\1/' -xvf '*.tar'
gtar: *.tar: Cannot open: No such file or directory
gtar: Error is not recoverable: exiting now
+ mkdir archive
mkdir: archive: File exists
+ mv '*.tar' archive/
mv: rename *.tar to archive/*.tar: No such file or directory


In [12]:
# util
def union_dicts(ds):
    '''
    example:
        union_dicts([{'a':1}, {'a':2, 'b':3}])
        {'a': 2, 'b': 3}
    '''
    d0 = {}
    [d0.update(d) for d in ds]
    return d0

In [13]:
RE_TS = re.compile('\d{8}_\d{6}')
get_ts = lambda x: RE_TS.search(x).group(0)
RE_PARAMS = re.compile('Trainable.*: (.*)')
get_params_from_line = lambda x: RE_PARAMS.search(x).group(1)

In [22]:
all_ts = [get_ts(x) for x in sh.ls('-l', ARCHIVE) if "hyperparameters.json" in x]
# all_ts[-1]

In [23]:
all_ts = [x for x in all_ts if '201705' in x]

In [24]:
# all_ts

In [25]:
# methods to query data from files in a meaningful way

get_hyp = lambda ts: json.load(open(ARCHIVE+ts+'_hyperparameters.json'))
get_archiver = lambda ts: json.load(open(ARCHIVE+ts+'_archiver.json'))
get_params = lambda ts: float(get_params_from_line(''.join(open(ARCHIVE+ts+'_summary.txt').readlines())).replace(',',''))
get_training_stats = lambda ts:pd.DataFrame.from_csv('./data/archive/'+ts+'_history.csv').describe()

In [26]:
for x in list(all_ts):
    try:
        get_training_stats(x)
    except Exception as e:
        all_ts.remove(x)

In [27]:
all_ts

[u'20170503_042715', u'20170503_052644']

In [31]:
# print get_hyp('20170412_205634')
print get_archiver('20170503_042715')
print get_hyp('20170503_042715')['dataset']
print get_params('20170503_042715')
print get_training_stats('20170503_042715')

{u'close': u'20170503_042715', u'init': u'20170503_033729', u'hostname': u'beirut', u'open': u'20170503_033729'}
twitter
150401.0
             acc        auc       loss    val_acc    val_auc   val_loss
count  70.000000  70.000000  70.000000  70.000000  70.000000  70.000000
mean    0.901786   0.954262   0.257064   0.740333   0.886480   0.522643
std     0.061439   0.071013   0.102526   0.039124   0.017443   0.028077
min     0.622490   0.513588   0.179123   0.692771   0.771365   0.489771
25%     0.894829   0.958600   0.198052   0.702811   0.884790   0.502984
50%     0.928213   0.980140   0.214085   0.737952   0.889919   0.514095
75%     0.936747   0.986453   0.270534   0.771084   0.893722   0.536019
max     0.950803   0.990721   0.711898   0.827309   0.899324   0.672242


In [32]:
get_all_info = lambda x: union_dicts([get_hyp(x), get_archiver(x), {u'params': get_params(x), u'ts':x}])
all_jsons = [get_all_info(x) for x in all_ts]
df = pd.DataFrame(all_jsons)
# filter_lengths

In [33]:
# print get_all_info('20170412_205634').keys()
all_columns = [u'vocab_size', u'dropout_rate', u'epsilon', u'kernel_l2_regularization',
               u'embeddingspath', u'beta_1', u'commit_hash', u'filters', u'close', u'open',
               u'kernel_sizes', u'beta_2', u'dimensions', u'decay', u'finetune', u'pooling',
               u'init', u'lr', u'params', u'x', u'maxlen', u'ts',u'dataset']
df = df.filter(items=all_columns, axis=1)
df = df[~df.commit_hash.isnull()]

In [34]:
df.commit_hash = df.commit_hash.apply(lambda x:str(x)[:7])

In [36]:
# df = pd.DataFrame([get_all_info('20170420_041706')])
# df.ts

In [38]:
df.loc[:,'max_val_auc'] = [get_training_stats(c).val_auc['max'] for c in df.ts]
df.loc[:,'comments'] = ''

In [40]:
def models2view(models, stream=sys.stdout):
    row_format=' '\
    '<tr>'\
    '		 <td width="25%">'\
                            'Comments: {comments}<br>'\
                            'Time stamp: {ts}<br>'\
                            'Trainable params: {params}<br>'\
                            'max_val_auc: {max_val_auc}<br>'\
                            '<iframe src="{ts}_hyperparameters.json"></iframe>'\
    '	</td><td width="25%"><img width="100%" src="{ts}_auc.png">'\
    '	</td><td width="25%"><img width="100%" src="{ts}_acc.png">'\
    '	</td><td width="25%"><img width="100%" src="{ts}_loss.png">'\
    '	</td>'\
    '</tr>'\
    '\n'
    stream.write( """<table style="width:100%">\n""")
    for m in models:
        stream.write(row_format.format(**m))
    stream.write( """</table>\n""")


In [41]:
df.head()

,vocab_size,dropout_rate,epsilon,kernel_l2_regularization,embeddingspath,beta_1,commit_hash,filters,close,open,...,pooling,init,lr,params,x,maxlen,ts,dataset,max_val_auc,comments
0,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,875aa18,50,20170503_042715,20170503_033729,...,logsumexp,20170503_033729,0.001,150401.0,4,1245,20170503_042715,twitter,0.899324,
1,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,875aa18,50,20170503_052644,20170503_050903,...,logsumexp,20170503_050903,0.001,150401.0,4,936,20170503_052644,yelp,0.895169,


In [44]:
models2view(df.to_dict(orient='record'), stream=open(ARCHIVE+'/v-yelp_vs_twitter.html', 'w'))

In [119]:
df.head()

g = df.groupby(['commit_hash', 'pooling']).aggregate({'close':(len,min)})
g.sort_values(by=[('close','min')])


idx1 = df.commit_hash=='200bc32'
idx2 = df.pooling=='logsumexp'
idx3 = df.kernel_l2_regularization==0.01

g = df[idx1 & idx2 & idx3].groupby(['pooling', 'filters', 'commit_hash']).aggregate({'close':(len,min,max), 'init':(min)})
g.unstack().fillna('')
# g.sort_values(by=[('close','min')])

# df[idx1 & idx2 & idx3].sort_values(by='filters')

g = df.groupby(['pooling', 'filters', 'kernel_l2_regularization','commit_hash']).aggregate({'close':(len,min,max)})
g.unstack().fillna('')

# idx1 = df1.close.apply(lambda x:x in ('20170411_015435','20170411_021851','20170411_044031','20170411_060103'))
# df1=df1[idx1|(df.commit_hash=='200bc32')]
# # df1=df1[df1.filters==5]
# # df1 = df[(df.commit_hash=='7543e17')&(df.kernel_l2_regularization==0.01)&(df.filters==5)]

# g = df1.groupby(['pooling', 'filters', 'kernel_l2_regularization','commit_hash']).aggregate({'close':(len,min,max)})
# g.unstack().fillna('')

# # df1
# # # for r in df1.to_dict(orient='records'):
# # #     print json.dumps(r, indent=2)

idx1 = df.commit_hash=='722ddec'
idx2 = df.commit_hash=='2b0f347'
df[idx1|idx2].groupby(['x']).aggregate({'close':(len,min,max)})

# df.close.apply(lambda x:x[:8])
g = df.groupby(['commit_hash']).aggregate({'close':(len,min,max)})
g.unstack().fillna('').sort_values()


            commit_hash
close  len  8d39d0e                      5
            4b7ce11                      8
       min  8d39d0e        20170420_032726
            4b7ce11        20170420_033333
       max  8d39d0e        20170420_041410
            4b7ce11        20170420_060824
dtype: object

In [42]:
# models2view(['20170412_011012'])
df1 = df[df.commit_hash == '3d4ce90']
df1 = df1.sort_values(by=['pooling', 'kernel_l2_regularization'])
df1.comments = [p+str(r) for p,r in zip(df1.pooling,df1.kernel_l2_regularization)]
models2view(df1.to_dict(orient='record'), stream=open(ARCHIVE+'/v-logsumexp_vs_max_pooling.html', 'w'))
df1

,vocab_size,dropout_rate,epsilon,kernel_l2_regularization,embeddingspath,beta_1,commit_hash,filters,close,open,...,decay,finetune,pooling,init,lr,params,x,maxlen,max_val_auc,comments
29,174716,0.5,1.000000e-08,0.000,/tmp/yo/foodborne/vectors_yelp.txt,0.9,3d4ce90,5.0,20170411_221343,20170411_215317,...,0.0,False,logsumexp,20170411_215317,0.001,9031.0,3,936,0.865729,logsumexp0.0
30,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,3d4ce90,5.0,20170411_223812,20170411_221356,...,0.0,False,logsumexp,20170411_221356,0.001,9031.0,3,936,0.897121,logsumexp0.001
32,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,3d4ce90,50.0,20170412_011012,20170411_234759,...,0.0,False,max,20170411_234758,0.001,90301.0,3,936,0.880205,max0.001
31,174716,0.5,1.000000e-08,0.010,/tmp/yo/foodborne/vectors_yelp.txt,0.9,3d4ce90,50.0,20170411_234744,20170411_223827,...,0.0,False,max,20170411_223827,0.001,90301.0,3,936,0.878985,max0.01


In [43]:
idx1 = df.commit_hash == '200bc32'
idx2 = df.params==9031
df1 = df[idx1 & idx2]
df1 = df1.sort_values(by='kernel_l2_regularization')
df1.comments = ['regularization='+str(x) for x in df1.kernel_l2_regularization]


models2view(df1.to_dict(orient='record'), stream=open(ARCHIVE+'/v-regularization.html', 'w'))
df1

,vocab_size,dropout_rate,epsilon,kernel_l2_regularization,embeddingspath,beta_1,commit_hash,filters,close,open,...,decay,finetune,pooling,init,lr,params,x,maxlen,max_val_auc,comments
19,174716,0.5,1.000000e-08,0.0000,/tmp/yo/foodborne/vectors_yelp.txt,0.9,200bc32,5.0,20170411_023338,20170411_011901,...,0.0,False,logsumexp,20170411_011901,0.001,9031.0,3,936,0.889639,regularization=0.0
28,174716,0.5,1.000000e-08,0.0001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,200bc32,5.0,20170411_073620,20170411_052107,...,0.0,False,logsumexp,20170411_052107,0.001,9031.0,3,936,0.859223,regularization=0.0001
26,174716,0.5,1.000000e-08,0.0010,/tmp/yo/foodborne/vectors_yelp.txt,0.9,200bc32,5.0,20170411_052047,20170411_034101,...,0.0,False,logsumexp,20170411_034101,0.001,9031.0,3,936,0.883946,regularization=0.001
22,174716,0.5,1.000000e-08,0.0100,/tmp/yo/foodborne/vectors_yelp.txt,0.9,200bc32,5.0,20170411_034042,20170411_023356,...,0.0,False,logsumexp,20170411_023356,0.001,9031.0,3,936,0.877358,regularization=0.01


In [44]:
idx1 = (df.commit_hash=='200bc32')|((df.commit_hash=='7543e17')&(df.filters==10))
idx2 = df.pooling=='logsumexp'
idx3 = df.kernel_l2_regularization==0.01

df1 = df[idx1 & idx2 & idx3]
df1 = df1.sort_values(by='filters')
df1.comments = ['filters='+str(x) for x in df1.filters]

models2view(df1.to_dict(orient='record'), stream=open(ARCHIVE+'/v-numfilters.html', 'w'))
g = df1.groupby(['pooling', 'filters', 'kernel_l2_regularization','commit_hash']).aggregate({'close':(len,min)})
g.unstack().fillna('')
# #.loc[:,('close','max')]

close                           \
                                               len                      min   
commit_hash                                200bc32 7543e17          200bc32   
pooling   filters kernel_l2_regularization                                    
logsumexp 5.0     0.01                           1          20170411_034042   
          10.0    0.01                                   1                    
          25.0    0.01                           1          20170411_021851   
          50.0    0.01                           1          20170411_044031   
          100.0   0.01                           1          20170411_060103   

                                                             
                                                             
commit_hash                                         7543e17  
pooling   filters kernel_l2_regularization                   
logsumexp 5.0     0.01                                       
          10.0    0.01                      20170411_015435  
          25.0    0.01                                       
          50.0    0.01                                       
          100.0   0.01

In [45]:
idx1 = df.commit_hash=='722ddec'
idx2 = df.commit_hash=='2b0f347'
df1 = df[idx1|idx2]
df1 = df1.sort_values(by='filters')
df1.comments = ['kernel_sizes='+str(x) for x in df1.kernel_sizes]

models2view(df1.to_dict(orient='record'), stream=open(ARCHIVE+'/v-ngrams.html', 'w'))


In [53]:
idx1 = df.commit_hash=='6d48133'
idx2 = df.pooling=='logsumexp'
df1 = df[idx1&idx2]
# df1 = df1.sort_values(by='filters')
df1.comments = ['kernel_sizes='+str(x) for x in df1.kernel_sizes]

models2view(df1.to_dict(orient='record'), stream=open(ARCHIVE+'/v1.html', 'w'))

df1

,vocab_size,dropout_rate,epsilon,kernel_l2_regularization,embeddingspath,beta_1,commit_hash,filters,close,open,...,decay,finetune,pooling,init,lr,params,x,maxlen,max_val_auc,comments
50,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,6d48133,5.0,20170419_180627,20170419_175723,...,0.0,False,logsumexp,20170419_175723,0.001,15041.0,4,936,0.886955,"kernel_sizes=[1, 2, 3, 4]"
51,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,6d48133,5.0,20170419_182228,20170419_180641,...,0.0,False,logsumexp,20170419_180640,0.001,9031.0,3,936,0.868738,"kernel_sizes=[1, 2, 3]"
52,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,6d48133,5.0,20170419_183211,20170419_182242,...,0.0,False,logsumexp,20170419_182242,0.001,4521.0,2,936,0.879961,"kernel_sizes=[1, 2]"


In [54]:

df[df.close == '20170412_205634'].loc[36]
df[df.commit_hash=='722ddec']

,vocab_size,dropout_rate,epsilon,kernel_l2_regularization,embeddingspath,beta_1,commit_hash,filters,close,open,...,decay,finetune,pooling,init,lr,params,x,maxlen,max_val_auc,comments
36,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,722ddec,5.0,20170412_205634,20170412_203050,...,0.0,False,logsumexp,20170412_203050,0.001,15041.0,4,936,0.894275,
37,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,722ddec,5.0,20170412_213719,20170412_205647,...,0.0,False,logsumexp,20170412_205647,0.001,22551.0,5,936,0.872398,


In [136]:
idx1 = df.commit_hash=='8d39d0e'
df1 = df[idx1]
# df1 = df1.sort_values(by='filters')
df1.comments = ['kernel_sizes='+str(x) for x in df1.kernel_sizes]

models2view(df1.to_dict(orient='record'), stream=open(ARCHIVE+'/v1.html', 'w'))

df1

,vocab_size,dropout_rate,epsilon,kernel_l2_regularization,embeddingspath,beta_1,commit_hash,filters,close,open,...,finetune,pooling,init,lr,params,x,maxlen,ts,max_val_auc,comments
0,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,8d39d0e,50,20170420_032726,20170420_031429,...,False,logsumexp,20170420_031429,0.001,150401.0,4,936,20170420_032726,0.879880,"kernel_sizes=[1, 2, 3, 4]"
2,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,8d39d0e,50,20170420_033759,20170420_032738,...,False,logsumexp,20170420_032738,0.001,90301.0,3,936,20170420_033759,0.872804,"kernel_sizes=[1, 2, 3]"
3,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,8d39d0e,50,20170420_034536,20170420_033812,...,False,logsumexp,20170420_033812,0.001,45201.0,2,936,20170420_034536,0.867762,"kernel_sizes=[1, 2]"
4,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,8d39d0e,50,20170420_035121,20170420_034551,...,False,logsumexp,20170420_034551,0.001,15101.0,1,936,20170420_035121,0.873130,kernel_sizes=[1]
6,174716,0.5,1.000000e-08,0.001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,8d39d0e,50,20170420_041410,20170420_035137,...,False,logsumexp,20170420_035137,0.001,225501.0,5,936,20170420_041410,0.885410,"kernel_sizes=[1, 2, 3, 4, 5]"


In [135]:
idx1 = df.commit_hash=='4b7ce11'
df1 = df[idx1]
# df1 = df1.sort_values(by='filters')
df1.comments = [str((x,y)) for (x,y) in zip(df1.pooling, df1.kernel_l2_regularization)]

models2view(df1.to_dict(orient='record'), stream=open(ARCHIVE+'/v-logsumexp_vs_max_pooling.html', 'a'))

df1

,vocab_size,dropout_rate,epsilon,kernel_l2_regularization,embeddingspath,beta_1,commit_hash,filters,close,open,...,finetune,pooling,init,lr,params,x,maxlen,ts,max_val_auc,comments
1,174716,0.5,1.000000e-08,0.0010,/tmp/yo/foodborne/vectors_yelp.txt,0.9,4b7ce11,50,20170420_033333,20170420_031857,...,False,logsumexp,20170420_031857,0.001,150401.0,4,936,20170420_033333,0.884190,"(u'logsumexp', 0.001)"
5,174716,0.5,1.000000e-08,0.0010,/tmp/yo/foodborne/vectors_yelp.txt,0.9,4b7ce11,50,20170420_040221,20170420_033346,...,False,max,20170420_033346,0.001,150401.0,4,936,20170420_040221,0.878822,"(u'max', 0.001)"
7,174716,0.5,1.000000e-08,0.0100,/tmp/yo/foodborne/vectors_yelp.txt,0.9,4b7ce11,50,20170420_041705,20170420_040234,...,False,logsumexp,20170420_040234,0.001,150401.0,4,936,20170420_041706,0.875163,"(u'logsumexp', 0.01)"
8,174716,0.5,1.000000e-08,0.0100,/tmp/yo/foodborne/vectors_yelp.txt,0.9,4b7ce11,50,20170420_044449,20170420_041720,...,False,max,20170420_041720,0.001,150401.0,4,936,20170420_044449,0.871503,"(u'max', 0.01)"
9,174716,0.5,1.000000e-08,0.0001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,4b7ce11,50,20170420_050022,20170420_044505,...,False,logsumexp,20170420_044505,0.001,150401.0,4,936,20170420_050022,0.885898,"(u'logsumexp', 0.0001)"
10,174716,0.5,1.000000e-08,0.0001,/tmp/yo/foodborne/vectors_yelp.txt,0.9,4b7ce11,50,20170420_052526,20170420_050040,...,False,max,20170420_050040,0.001,150401.0,4,936,20170420_052526,0.867355,"(u'max', 0.0001)"
11,174716,0.5,1.000000e-08,0.0000,/tmp/yo/foodborne/vectors_yelp.txt,0.9,4b7ce11,50,20170420_054005,20170420_052544,...,False,logsumexp,20170420_052544,0.001,150401.0,4,936,20170420_054005,0.890208,"(u'logsumexp', 0.0)"
12,174716,0.5,1.000000e-08,0.0000,/tmp/yo/foodborne/vectors_yelp.txt,0.9,4b7ce11,50,20170420_060824,20170420_054026,...,False,max,20170420_054026,0.001,150401.0,4,936,20170420_060824,0.877359,"(u'max', 0.0)"


In [128]:
# df